In [2]:
import pandas as pd
import numpy as np
import random

# Load the existing data
data = pd.read_csv('game_data.csv')

# Current number of rows
current_rows = len(data)
target_rows = 20000
rows_to_add = target_rows - current_rows

# Analyze patterns for generation
initial_health = 176
initial_timer = 150
players = [1, 2]  # Allow different player IDs for variety
move_ids = [0, 1, 131073, 33686532, 67240965, 132100]  # Observed move IDs

# Columns for the new data
columns = data.columns

# Function to generate a single round (150 to 1 timer)
def generate_round(round_health_p1, round_health_p2, player1_id, player2_id):
    round_data = []
    p1_x, p1_y = 205, 192  # Initial positions
    p2_x, p2_y = 339, 192
    health_p1, health_p2 = round_health_p1, round_health_p2
    
    for timer in range(150, 0, -1):
        # Fight result and round state
        fight_result = "NOT_OVER"
        if timer == 1 and (health_p1 <= 0 or health_p2 <= 0):
            fight_result = "PLAYER1_WINS" if health_p2 <= 0 else "PLAYER2_WINS"
        
        # Simulate movement and actions
        p1_buttons = {
            'up': random.choice([True, False]),
            'down': random.choice([True, False]),
            'right': random.choice([True, False]),
            'left': random.choice([True, False])
        }
        p2_buttons = {
            'up': random.choice([True, False]),
            'down': random.choice([True, False]),
            'right': random.choice([True, False]),
            'left': random.choice([True, False])
        }
        
        # Update positions based on buttons
        if p1_buttons['right'] and not p1_buttons['left']:
            p1_x = min(p1_x + random.randint(1, 5), 400)  # Max x boundary
        elif p1_buttons['left'] and not p1_buttons['right']:
            p1_x = max(p1_x - random.randint(1, 5), 0)  # Min x boundary
        
        if p2_buttons['right'] and not p2_buttons['left']:
            p2_x = min(p2_x + random.randint(1, 5), 400)
        elif p2_buttons['left'] and not p2_buttons['right']:
            p2_x = max(p2_x - random.randint(1, 5), 0)
        
        # Jumping and crouching
        p1_jumping = p1_buttons['up']
        p1_crouching = p1_buttons['down']
        p2_jumping = p2_buttons['up']
        p2_crouching = p2_buttons['down']
        
        p1_y = 192 if not p1_jumping else random.randint(100, 192)
        p2_y = 192 if not p2_jumping else random.randint(100, 192)
        
        # Player in move
        p1_in_move = random.choice([True, False])
        p2_in_move = random.choice([True, False])
        p1_move_id = random.choice(move_ids) if p1_in_move else 0
        p2_move_id = random.choice(move_ids) if p2_in_move else 0
        
        # Health decrease (simulate hits)
        if p1_in_move and abs(p1_x - p2_x) < 50 and random.random() < 0.1:
            health_p2 = max(health_p2 - random.randint(1, 5), 0)
        if p2_in_move and abs(p1_x - p2_x) < 50 and random.random() < 0.1:
            health_p1 = max(health_p1 - random.randint(1, 5), 0)
        
        # Create row
        row = {
            'timer': timer,
            'fight_result': fight_result,
            'has_round_started': True,
            'is_round_over': timer == 1,
            'Player1_ID': player1_id,
            'health': health_p1,
            'x_coord': p1_x,
            'y_coord': p1_y,
            'is_jumping': p1_jumping,
            'is_crouching': p1_crouching,
            'is_player_in_move': p1_in_move,
            'move_id': p1_move_id,
            'player1_buttons up': p1_buttons['up'],
            'player1_buttons down': p1_buttons['down'],
            'player1_buttons right': p1_buttons['right'],
            'player1_buttons left': p1_buttons['left'],
            'Player2_ID': player2_id,
            'Player2 health': health_p2,
            'Player2 x_coord': p2_x,
            'Player2 y_coord': p2_y,
            'Player2 is_jumping': p2_jumping,
            'Player2 is_crouching': p2_crouching,
            'Player2 is_player_in_move': p2_in_move,
            'Player2 move_id': p2_move_id,
            'player2_buttons up': p2_buttons['up'],
            'player2_buttons down': p2_buttons['down'],
            'player2_buttons right': p2_buttons['right'],
            'player2_buttons left': p2_buttons['left']
        }
        round_data.append(row)
    
    return round_data, health_p1, health_p2

# Generate new data
new_data = []
health_p1, health_p2 = initial_health, initial_health
player1_id, player2_id = 2, 2  # Start with same IDs as original data

while len(new_data) < rows_to_add:
    round_data, health_p1, health_p2 = generate_round(health_p1, health_p2, player1_id, player2_id)
    new_data.extend(round_data)
    
    # Reset health and possibly change player IDs for the next round
    health_p1, health_p2 = initial_health, initial_health
    player1_id = random.choice(players)
    player2_id = random.choice(players)
    
    # Trim excess rows
    if len(new_data) > rows_to_add:
        new_data = new_data[:rows_to_add]

# Convert new data to DataFrame
new_df = pd.DataFrame(new_data, columns=columns)

# Append to original data
expanded_data = pd.concat([data, new_df], ignore_index=True)

# Save to CSV
expanded_data.to_csv('expanded_game_data.csv', index=False)
print(f"Expanded dataset to {len(expanded_data)} rows and saved to 'expanded_game_data.csv'.")

Expanded dataset to 20000 rows and saved to 'expanded_game_data.csv'.


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import joblib

# Load the dataset
data = pd.read_csv('expanded_game_data.csv')

# Feature Engineering
data['relative_distance'] = np.abs(data['x_coord'] - data['Player2 x_coord'])
data['p1_x_velocity'] = data['x_coord'].diff().fillna(0)
data['p1_y_velocity'] = data['y_coord'].diff().fillna(0)
data['p2_x_velocity'] = data['Player2 x_coord'].diff().fillna(0)
data['p2_y_velocity'] = data['Player2 y_coord'].diff().fillna(0)
data['health_diff'] = data['health'] - data['Player2 health']

# Define features and labels
feature_columns = [
    'health', 'x_coord', 'y_coord', 'is_jumping', 'is_crouching',
    'is_player_in_move', 'move_id', 'Player2 health', 'Player2 x_coord',
    'Player2 y_coord', 'Player2 is_jumping', 'Player2 is_crouching',
    'Player2 is_player_in_move', 'Player2 move_id', 'relative_distance',
    'p1_x_velocity', 'p1_y_velocity', 'p2_x_velocity', 'p2_y_velocity',
    'health_diff'
]
label_columns = [
    'player1_buttons up', 'player1_buttons down',
    'player1_buttons right', 'player1_buttons left'
]

X = data[feature_columns]
y = data[label_columns]

# Convert boolean columns to integers
X = X.astype({col: int for col in X.columns if X[col].dtype == bool})
y = y.astype(int)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler
joblib.dump(scaler, 'scaler_tuned.pkl')

# Define the MLP model
mlp = MLPClassifier(
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=False
)

# Wrap in MultiOutputClassifier
multi_label_model = MultiOutputClassifier(mlp, n_jobs=-1)

# Define parameter grid for GridSearchCV
param_grid = {
    'estimator__hidden_layer_sizes': [(128, 64, 32), (256, 128, 64)],
    'estimator__learning_rate_init': [0.0005, 0.001],
    'estimator__alpha': [0.0001, 0.001]  # L2 regularization
}

# Perform GridSearchCV
grid_search = GridSearchCV(
    multi_label_model,
    param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train_scaled, y_train)

# Best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Save the best model
joblib.dump(best_model, 'mlp_model_tuned.pkl')

# Predict on test set
y_pred = best_model.predict(X_test_scaled)

# Evaluate the model
print("\nEvaluation Metrics:")
overall_accuracy = accuracy_score(y_test, y_pred)
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Per-label and macro-averaged metrics
precisions, recalls, f1_scores, _ = precision_recall_fscore_support(
    y_test, y_pred, average=None, labels=[0, 1, 2, 3]
)
macro_f1 = precision_recall_fscore_support(
    y_test, y_pred, average='macro'
)[2]

for i, label in enumerate(label_columns):
    print(f"\nLabel: {label}")
    print(f"Precision: {precisions[i]:.4f}")
    print(f"Recall: {recalls[i]:.4f}")
    print(f"F1-Score: {f1_scores[i]:.4f}")

print(f"\nMacro-Averaged F1-Score: {macro_f1:.4f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Parameters: {'estimator__alpha': 0.0001, 'estimator__hidden_layer_sizes': (256, 128, 64), 'estimator__learning_rate_init': 0.001}

Evaluation Metrics:
Overall Accuracy: 0.6415

Label: player1_buttons up
Precision: 0.9994
Recall: 0.9883
F1-Score: 0.9938

Label: player1_buttons down
Precision: 0.9754
Recall: 0.9094
F1-Score: 0.9413

Label: player1_buttons right
Precision: 0.7432
Recall: 0.7198
F1-Score: 0.7313

Label: player1_buttons left
Precision: 0.7597
Recall: 0.7004
F1-Score: 0.7289

Macro-Averaged F1-Score: 0.8488


In [7]:
import pandas as pd
import numpy as np
import joblib

# Load the trained model and scaler
model = joblib.load('mlp_model_tuned.pkl')
scaler = joblib.load('scaler_tuned.pkl')

# Define feature columns (must match training)
feature_columns = [
    'health', 'x_coord', 'y_coord', 'is_jumping', 'is_crouching',
    'is_player_in_move', 'move_id', 'Player2 health', 'Player2 x_coord',
    'Player2 y_coord', 'Player2 is_jumping', 'Player2 is_crouching',
    'Player2 is_player_in_move', 'Player2 move_id', 'relative_distance',
    'p1_x_velocity', 'p1_y_velocity', 'p2_x_velocity', 'p2_y_velocity',
    'health_diff'
]

# Function to predict buttons from game state
def predict_buttons(game_state):
    input_data = pd.DataFrame([game_state], columns=feature_columns)
    input_data = input_data.astype({col: int for col in input_data.columns if input_data[col].dtype == bool})
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    label_columns = [
        'player1_buttons up', 'player1_buttons down',
        'player1_buttons right', 'player1_buttons left'
    ]
    return dict(zip(label_columns, prediction))

# Test Case 1: Original state (moderate distance, no velocity)
state_1 = {
    'health': 150,
    'x_coord': 205,
    'y_coord': 192,
    'is_jumping': False,
    'is_crouching': False,
    'is_player_in_move': True,
    'move_id': 131073,
    'Player2 health': 120,
    'Player2 x_coord': 339,
    'Player2 y_coord': 192,
    'Player2 is_jumping': False,
    'Player2 is_crouching': False,
    'Player2 is_player_in_move': True,
    'Player2 move_id': 33686532,
    'relative_distance': abs(205 - 339),
    'p1_x_velocity': 0,
    'p1_y_velocity': 0,
    'p2_x_velocity': 0,
    'p2_y_velocity': 0,
    'health_diff': 150 - 120
}

# Test Case 2: Players closer, Player 1 moving toward Player 2
state_2 = {
    'health': 150,
    'x_coord': 290,
    'y_coord': 192,
    'is_jumping': False,
    'is_crouching': False,
    'is_player_in_move': True,
    'move_id': 0,
    'Player2 health': 120,
    'Player2 x_coord': 339,
    'Player2 y_coord': 192,
    'Player2 is_jumping': False,
    'Player2 is_crouching': False,
    'Player2 is_player_in_move': False,
    'Player2 move_id': 0,
    'relative_distance': abs(290 - 339),
    'p1_x_velocity': 5,  # Moving right
    'p1_y_velocity': 0,
    'p2_x_velocity': 0,
    'p2_y_velocity': 0,
    'health_diff': 150 - 120
}

# Test Case 3: Player 1 jumping, Player 2 crouching
state_3 = {
    'health': 150,
    'x_coord': 205,
    'y_coord': 150,
    'is_jumping': True,
    'is_crouching': False,
    'is_player_in_move': True,
    'move_id': 0,
    'Player2 health': 120,
    'Player2 x_coord': 339,
    'Player2 y_coord': 192,
    'Player2 is_jumping': False,
    'Player2 is_crouching': True,
    'Player2 is_player_in_move': False,
    'Player2 move_id': 0,
    'relative_distance': abs(205 - 339),
    'p1_x_velocity': 0,
    'p1_y_velocity': -5,  # Jumping (y decreases)
    'p2_x_velocity': 0,
    'p2_y_velocity': 0,
    'health_diff': 150 - 120
}

# Predict for all test cases
test_cases = [state_1, state_2, state_3]
for i, state in enumerate(test_cases, 1):
    print(f"\nTest Case {i}:")
    print("Game State:", {k: v for k, v in state.items() if k in ['x_coord', 'Player2 x_coord', 'relative_distance', 'p1_x_velocity', 'is_jumping', 'is_crouching']})
    result = predict_buttons(state)
    print("Predicted Button States:")
    for button, value in result.items():
        print(f"{button}: {value}")


Test Case 1:
Game State: {'x_coord': 205, 'is_jumping': False, 'is_crouching': False, 'Player2 x_coord': 339, 'relative_distance': 134, 'p1_x_velocity': 0}
Predicted Button States:
player1_buttons up: 0
player1_buttons down: 0
player1_buttons right: 0
player1_buttons left: 0

Test Case 2:
Game State: {'x_coord': 290, 'is_jumping': False, 'is_crouching': False, 'Player2 x_coord': 339, 'relative_distance': 49, 'p1_x_velocity': 5}
Predicted Button States:
player1_buttons up: 0
player1_buttons down: 0
player1_buttons right: 1
player1_buttons left: 0

Test Case 3:
Game State: {'x_coord': 205, 'is_jumping': True, 'is_crouching': False, 'Player2 x_coord': 339, 'relative_distance': 134, 'p1_x_velocity': 0}
Predicted Button States:
player1_buttons up: 1
player1_buttons down: 0
player1_buttons right: 0
player1_buttons left: 0


In [8]:
import pandas as pd
import numpy as np
import joblib
import time

# Load the trained model and scaler
model = joblib.load('mlp_model_tuned.pkl')
scaler = joblib.load('scaler_tuned.pkl')

# Define feature columns
feature_columns = [
    'health', 'x_coord', 'y_coord', 'is_jumping', 'is_crouching',
    'is_player_in_move', 'move_id', 'Player2 health', 'Player2 x_coord',
    'Player2 y_coord', 'Player2 is_jumping', 'Player2 is_crouching',
    'Player2 is_player_in_move', 'Player2 move_id', 'relative_distance',
    'p1_x_velocity', 'p1_y_velocity', 'p2_x_velocity', 'p2_y_velocity',
    'health_diff'
]

# Function to predict buttons
def predict_buttons(game_state):
    input_data = pd.DataFrame([game_state], columns=feature_columns)
    input_data = input_data.astype({col: int for col in input_data.columns if input_data[col].dtype == bool})
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    label_columns = [
        'player1_buttons up', 'player1_buttons down',
        'player1_buttons right', 'player1_buttons left'
    ]
    return dict(zip(label_columns, prediction))

# Initial game state
game_state = {
    'health': 150,
    'x_coord': 205,
    'y_coord': 192,
    'is_jumping': False,
    'is_crouching': False,
    'is_player_in_move': True,
    'move_id': 131073,
    'Player2 health': 120,
    'Player2 x_coord': 339,
    'Player2 y_coord': 192,
    'Player2 is_jumping': False,
    'Player2 is_crouching': False,
    'Player2 is_player_in_move': True,
    'Player2 move_id': 33686532,
    'relative_distance': abs(205 - 339),
    'p1_x_velocity': 0,
    'p1_y_velocity': 0,
    'p2_x_velocity': 0,
    'p2_y_velocity': 0,
    'health_diff': 150 - 120
}

# Simulate game loop
print("Starting real-time simulation (press Ctrl+C to stop)...")
try:
    while True:
        # Update game state (simulated random changes)
        prev_x = game_state['x_coord']
        game_state['x_coord'] += np.random.randint(-5, 6)  # Random movement
        game_state['y_coord'] = 192 if not game_state.get('is_jumping', False) else np.random.randint(100, 192)
        game_state['is_jumping'] = np.random.choice([True, False], p=[0.2, 0.8])
        game_state['is_crouching'] = np.random.choice([True, False], p=[0.2, 0.8])
        game_state['is_player_in_move'] = np.random.choice([True, False])
        game_state['move_id'] = np.random.choice([0, 131073, 33686532])
        game_state['Player2 x_coord'] += np.random.randint(-5, 6)
        game_state['Player2 y_coord'] = 192 if not game_state.get('Player2 is_jumping', False) else np.random.randint(100, 192)
        game_state['Player2 is_player_in_move'] = np.random.choice([True, False])
        game_state['Player2 move_id'] = np.random.choice([0, 131073, 33686532])

        # Update derived features
        game_state['relative_distance'] = abs(game_state['x_coord'] - game_state['Player2 x_coord'])
        game_state['p1_x_velocity'] = game_state['x_coord'] - prev_x
        game_state['p1_y_velocity'] = (game_state['y_coord'] - 192) if not game_state['is_jumping'] else 0
        game_state['p2_x_velocity'] = game_state['Player2 x_coord'] - (prev_x + 134)  # Approximate
        game_state['p2_y_velocity'] = 0
        game_state['health_diff'] = game_state['health'] - game_state['Player2 health']

        # Predict and print
        prediction = predict_buttons(game_state)
        print("\nGame State (x, Player2 x, distance, p1_x_vel):",
              f"{game_state['x_coord']}, {game_state['Player2 x_coord']}, {game_state['relative_distance']}, {game_state['p1_x_velocity']}")
        print("Predicted Button States:")
        for button, value in prediction.items():
            print(f"{button}: {value}")

        time.sleep(1)  # Simulate 1 second per frame

except KeyboardInterrupt:
    print("\nSimulation stopped by user.")

Starting real-time simulation (press Ctrl+C to stop)...

Game State (x, Player2 x, distance, p1_x_vel): 205, 343, 138, 0
Predicted Button States:
player1_buttons up: 0
player1_buttons down: 0
player1_buttons right: 0
player1_buttons left: 0

Game State (x, Player2 x, distance, p1_x_vel): 202, 339, 137, -3
Predicted Button States:
player1_buttons up: 0
player1_buttons down: 0
player1_buttons right: 1
player1_buttons left: 0

Game State (x, Player2 x, distance, p1_x_vel): 199, 334, 135, -3
Predicted Button States:
player1_buttons up: 1
player1_buttons down: 0
player1_buttons right: 0
player1_buttons left: 0

Game State (x, Player2 x, distance, p1_x_vel): 204, 335, 131, 5
Predicted Button States:
player1_buttons up: 0
player1_buttons down: 0
player1_buttons right: 1
player1_buttons left: 0

Game State (x, Player2 x, distance, p1_x_vel): 209, 333, 124, 5
Predicted Button States:
player1_buttons up: 0
player1_buttons down: 0
player1_buttons right: 0
player1_buttons left: 0

Game State (x, P

In [9]:
import os

def list_all_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            full_path = os.path.join(root, file)
            print(full_path)

if __name__ == "__main__":
    directory = "D:/AI Project (1)/AI Project/gamebot-competition-master"
    list_all_files(directory)

D:/AI Project (1)/AI Project/gamebot-competition-master\Controls.PNG
D:/AI Project (1)/AI Project/gamebot-competition-master\current_game_state0.txt
D:/AI Project (1)/AI Project/gamebot-competition-master\README.md
D:/AI Project (1)/AI Project/gamebot-competition-master\snes_controller_layout.png
D:/AI Project (1)/AI Project/gamebot-competition-master\JavaAPI\Bot.java
D:/AI Project (1)/AI Project/gamebot-competition-master\JavaAPI\Buttons.java
D:/AI Project (1)/AI Project/gamebot-competition-master\JavaAPI\Command.java
D:/AI Project (1)/AI Project/gamebot-competition-master\JavaAPI\Controller.java
D:/AI Project (1)/AI Project/gamebot-competition-master\JavaAPI\GameState.java
D:/AI Project (1)/AI Project/gamebot-competition-master\JavaAPI\Player.java
D:/AI Project (1)/AI Project/gamebot-competition-master\JavaAPI\lib\json-20160212.jar
D:/AI Project (1)/AI Project/gamebot-competition-master\PythonAPI\bot.py
D:/AI Project (1)/AI Project/gamebot-competition-master\PythonAPI\buttons.py
D:/A